## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210323.ods'

In [5]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210323.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [6]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,899820,111200,343800,1354820,1202553,0.887611,422339,2021-03-22
1,Aragón,165285,21700,54000,240985,207118,0.859464,72085,2021-03-22
2,Asturias,161195,20700,41500,223395,187429,0.839003,74197,2021-03-22
3,Baleares,86940,11100,47500,145540,124791,0.857434,35918,2021-03-22
4,Canarias,186000,22900,88400,297300,241508,0.812338,81868,2021-03-22
5,Cantabria,79125,9700,23600,112425,91015,0.809562,36503,2021-03-22
6,Castilla y Leon,364755,47700,97300,509755,431153,0.845804,157258,2021-03-22
7,Castilla La Mancha,239715,31000,83100,353815,271647,0.767766,94751,2021-03-22
8,Cataluña,819620,105400,316100,1241120,1011890,0.815304,346521,2021-03-22
9,C. Valenciana,450640,57800,205400,713840,569061,0.797183,201548,2021-03-22


..hacemos una copia para hacerle las transformaciones necesarias:

In [7]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [8]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [9]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [10]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [11]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [12]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,899820,111200,343800,1354820,1202553,0.887611,422339,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,165285,21700,54000,240985,207118,0.859464,72085,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,161195,20700,41500,223395,187429,0.839003,74197,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,86940,11100,47500,145540,124791,0.857434,35918,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,186000,22900,88400,297300,241508,0.812338,81868,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,79125,9700,23600,112425,91015,0.809562,36503,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,364755,47700,97300,509755,431153,0.845804,157258,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,239715,31000,83100,353815,271647,0.767766,94751,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,819620,105400,316100,1241120,1011890,0.815304,346521,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,450640,57800,205400,713840,569061,0.797183,201548,22/3/21,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [13]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [14]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [15]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [16]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,23/3/21,Andalucía,899820,111200,343800,1354820,1202553,0.887611,422339,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,23/3/21,Aragón,165285,21700,54000,240985,207118,0.859464,72085,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,23/3/21,Asturias,161195,20700,41500,223395,187429,0.839003,74197,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,23/3/21,Baleares,86940,11100,47500,145540,124791,0.857434,35918,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,23/3/21,Canarias,186000,22900,88400,297300,241508,0.812338,81868,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,23/3/21,Cantabria,79125,9700,23600,112425,91015,0.809562,36503,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,23/3/21,Castilla y Leon,364755,47700,97300,509755,431153,0.845804,157258,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,23/3/21,Castilla La Mancha,239715,31000,83100,353815,271647,0.767766,94751,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,23/3/21,Cataluña,819620,105400,316100,1241120,1011890,0.815304,346521,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,23/3/21,C. Valenciana,450640,57800,205400,713840,569061,0.797183,201548,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [17]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [18]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [19]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,22/3/21,Navarra,76785.0,9700.0,26800.0,113285,99308,0.876621,30828.0,NaN,21/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1076,22/3/21,País Vasco,238500.0,30800.0,90200.0,359500,262965,0.731474,72611.0,NaN,21/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1077,22/3/21,Ceuta,5490.0,900.0,3500.0,9890,9432,0.953691,2542.0,NaN,17/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1078,22/3/21,Melilla,4320.0,800.0,3500.0,8620,8773,1.017749,2384.0,NaN,20/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [20]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [21]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [22]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [23]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,23/3/21,Navarra,76785.0,9700.0,26800.0,113285,100046,0.883135,31120.0,NaN,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1096,23/3/21,País Vasco,238500.0,30800.0,90200.0,359500,277455,0.771780,75513.0,NaN,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1097,23/3/21,Ceuta,5490.0,900.0,3500.0,9890,9432,0.953691,2542.0,NaN,17/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1098,23/3/21,Melilla,4320.0,800.0,3500.0,8620,8937,1.036775,2387.0,NaN,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [24]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [25]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)